In [36]:
import numpy as np
import scipy.linalg as la

**1**.(20 points)

Consider the following system of equations:

$$\begin{align*}
2x_1& - x_2&    +x_3    &=& 6\\
-x_1& +2x_2& -  x_3 &=& 2\\
 x_1   &  -x_2& + x_3 &=& 1
\end{align*}$$

1. Consider the system in matrix form $Ax=b$ and define $A$, $b$ in numpy. (5 points)
2. Show that $A$ is positive-definite (5 points)
3. Use the appropriate matrix decomposition function in numpy and back-substitution to solve the system (10 points)

In [7]:
# Define A and b
A = np.array([2, -1, 1, -1, 2, -1, 1, -1, 1]).reshape(3, 3)
b = np.array([6, 2, 1]).reshape(-1, 1)

# Print A and b
print(A)
print(b)

[[ 2 -1  1]
 [-1  2 -1]
 [ 1 -1  1]]
[[6]
 [2]
 [1]]


*We verify that A is positive definite by (1) visually confirming that it is a symmetric matrix and (2) showing that all three eigenvalues of A are positive.*

In [9]:
np.linalg.eigvals(A)

array([3.73205081, 1.        , 0.26794919])

*Since A is positive definite, we can use the Cholesky decomposition to solve the system of equations.*

*After obtaining the Cholesky decomposition, A = CC', we can solve the following two equations in order using backwards substitution:*

- Cy = b
- C'x = y

*We show these 3 steps in order below*

In [15]:
# Find C
C = np.linalg.cholesky(A)
C

array([[ 1.41421356,  0.        ,  0.        ],
       [-0.70710678,  1.22474487,  0.        ],
       [ 0.70710678, -0.40824829,  0.57735027]])

In [17]:
# Solve Cy = b
y = np.linalg.solve(C, b)
y

array([[ 4.24264069],
       [ 4.0824829 ],
       [-0.57735027]])

In [18]:
# Solve C'x = y, which returns the result of the original system
np.linalg.solve(C.T, y)

array([[ 5.],
       [ 3.],
       [-1.]])

*We can check our answer by simply solving the original system without the matrix decomposition*

In [14]:
np.linalg.solve(A, b)

array([[ 5.],
       [ 3.],
       [-1.]])

*In the event that the backsubstitution is meant to be manual (i.e., not using a solve function), we can do so as follows*

In [35]:
# print C for reference
print(C)

# Initialize and solve for y (from Cy = b)
y = np.zeros(A.shape[0]).reshape(-1, 1)
y[0] = b[0] / C[0, 0]
y[1] = (b[1] - C[1, 0] * y[0]) / C[1, 1]
y[2] = (b[2] - C[2, 0] * y[0] - C[2, 1] * y[1]) / C[2, 2]
y

[[ 1.41421356  0.          0.        ]
 [-0.70710678  1.22474487  0.        ]
 [ 0.70710678 -0.40824829  0.57735027]]


array([[ 4.24264069],
       [ 4.0824829 ],
       [-0.57735027]])

In [33]:
# Print C' for reference
print(C.T)

# Initialize and solve for x (from C'x = y)
x = np.zeros(A.shape[0]).reshape(-1, 1)
x[2] = y[2] / C.T[2, 2]
x[1] = (y[1] - C.T[1, 2] * x[2]) / C.T[1, 1]
x[0] = (y[0] - C.T[0, 2] * x[2] - C.T[0, 1] * x[1]) / C[0, 0]
x

[[ 1.41421356 -0.70710678  0.70710678]
 [ 0.          1.22474487 -0.40824829]
 [ 0.          0.          0.57735027]]


array([[ 5.],
       [ 3.],
       [-1.]])

**2**. (20 points)

Exact geometric solutions with $n = m$

- Find the equation of the line that passes through the points (2,1) and (3,7)
- Find the equation of the circle that passes through the points (1,7), (6,2) and (4,6)

Hint: The equation of a circle can be written as

$$
(x - a)^2 + (y - b)^2 = r^2
$$

**3**. 20 points

- Load the matrix in `Q2.npy` - this consists of two columns representing the x and y coordinates of 10 points
- Find the equation of the circle that best fits these points
- Plot the points and fitted circle

Hint: You need to estimate the center of the circle and its radius.

**4**. 20 points

The figure below shows the current population of Durham, Chapel Hill and Raleigh. Arrows show fractions that move between cities each year.

- What are the population sizes of the 3 cities after 3 years have passed?
- Find the steady state population of the 3 cities by solving a linear system.

Assume no births, deaths or any other fluxes other than those shown.

![img](Q3.png)

**5** (20 points)

The file `Q5.npy` contains the x and y coordinates in cols 1 and 2 respectively. 

- Find a cubic polynomial model to fit the data using the normal equations
- Provide a geometric interpretation of the solution in terms of projection of a vector onto a space. What is the vector, what is the basis of the space, and what does the numerical solution you obtained represent?